In [1]:
import mosek
import numpy as np
from mosek import *
import mosek.fusion
from mosek.fusion import *
from math import *
from numpy import *
from numpy.random import *
import sys
import csv


In [13]:
#Parámetros

#Vector de escenarios
muestra=poisson(20,50)
Di=muestra.tolist()
Di.sort()

D=[Di[0]]
Pt=ones(size(Di))/size(Di)
P=[Pt[0]]
j=0

for i in range(1,size(Di)):
    if Di[i]!=D[j]:
        D.append(Di[i])
        P.append(Pt[i])
        j=j+1
    else:
        P[j]=P[j]+Pt[i]

        
s=size(D)


#c: Costo del producto & p: precio
c=5
p=12


eps=0.1
inf=10000
numeps=5


In [16]:
print(D)
print(size(D))

with open('escenarios.csv','w',newline='') as tabla: 
    thewriter=csv.writer(tabla)
    thewriter.writerow(D)
    thewriter.writerow(P)
    

[9, 11, 12, 13, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 31]
18


In [17]:
def gamma(A):
    probA=0 
    for i in range(size(A)):
        cual=D.index(A[i])
        probA=probA+P[cual] 
        
    gam=2*(1-probA+eps*probA)
    return gam

In [18]:
#Función que retorna el valor óptimo del problema newsvendor no robusto 

def objetivo(x,D,P):
    suma=0
    for i in range(size(D)):
        suma=suma+(min(D[i],x))*P[i]
        
    A=c*x-p*suma
    return A



In [19]:
#Definición de la función f(x,A)

def funcionf(x,A):

    s=size(D)
    
    probA=0 
    for i in range(size(A)):
        cual=D.index(A[i])
        probA=probA+P[cual]
        
    indicA=zeros(s)
    
    Q=zeros(s)
    for i in range(size(A)):
        cual=D.index(A[i])
        indicA[cual]=1
        Q[cual]=P[cual]/probA
    O=Q.tolist()
    
    O=P
    epsilon=gamma(A)
    
    matrizA=ones((3*s,3+2*s))

    with mosek.Env() as env:
            with env.Task(0, 0) as task:

                # Bound keys for constraints
                bkc = [mosek.boundkey.lo]*3*s   

                # Bound values for constraints
                menosPYs=[]
                for i in range(s):
                    menosPYs.append(-p*min(x,D[i])*indicA[i])

                blc = menosPYs+[0]*s*2
                buc = [inf]*3*s

                # Bound keys for variables
                bkx = [mosek.boundkey.fr]*2+[mosek.boundkey.lo]*(2*s+1) 

                # Bound values for variables
                blx = [-inf]*2+[0]*(2*s+1)
                bux = [+inf]*(3+2*s)

                # Objective coefficients
                #coef5=round(epsilon*s/size(A),0)
                mO=[]
                for i in range(size(O)):
                    mO.append(-O[i])

                coef = [1]+[0]+[epsilon]+O+mO
                
                task.putcfix(c*x)

                # Below is the sparse representation of the A
                # matrix stored by column.

                numvar = len(bkx)
                numcon = len(bkc)

                # Append 'numcon' empty constraints.
                # The constraints will initially have no bounds.
                task.appendcons(numcon)

                # Append 'numvar' variables.
                # The variables will initially be fixed at zero (x=0).
                task.appendvars(numvar)

                #Append constraint matrix 
                task.putaijlist(range(0,s),[0]*s,indicA)
                task.putaijlist(range(0,s),[1]*s,(ones(s)-indicA))
                task.putaijlist(range(0,s),range(3,3+s),ones(s))
                task.putaijlist(range(0,s),range(3+s,3+2*s),-ones(s))
                task.putaijlist(range(s,2*s),[2]*s,ones(s))
                task.putaijlist(range(s,2*s),range(3,3+s),-ones(s))
                task.putaijlist(range(2*s,3*s),[2]*s,ones(s))
                task.putaijlist(range(2*s,3*s),range(3+s,3+2*s),-ones(s))

                for i in range(numcon):
                    for j in range(numvar):
                        matrizA[i,j]=task.getaij(i,j) 


                for j in range(numvar):
                    # Set the linear term c_j in the objective.
                    task.putcj(j, coef[j])

                    # Set the bounds on variable j
                    # blx[j] <= x_j <= bux[j]
                    task.putvarbound(j, bkx[j], blx[j], bux[j])  


                # Set the bounds on constraints.
                # blc[i] <= constraint_i <= buc[i]
                for i in range(numcon):
                    task.putconbound(i, bkc[i], blc[i], buc[i])

                # Input the objective sense (minimize/maximize)
                task.putobjsense(mosek.objsense.minimize)

                # Solve the problem
                task.optimize()

                # Print a summary containing information
                # about the solution for debugging purposes
                resumen=task.solutionsummary(mosek.streamtype.msg)

                # Get status information about the solution
                solsta = task.getsolsta(mosek.soltype.bas)

                if (solsta == mosek.solsta.optimal or solsta == mosek.solsta.near_optimal): 
                    sol=task.getprimalobj(soltype.bas)
                elif (solsta == mosek.solsta.dual_infeas_cer ):
                    print("Dual infeasibility certificate found.\n")
                    sol="No factible"
                elif(solsta == mosek.solsta.prim_infeas_cer or solsta == mosek.solsta.near_prim_infeas_cer):
                    print("Primal infeasibility certificate found.\n")
                    sol="No factible"
                elif solsta == mosek.solsta.unknown:
                    print("Unknown solution status")
                    sol="No factible"
                else:
                    print("Other solution status")
                    sol="Other solution status"
    
    return sol


In [20]:
#Función que devuelve la solución x_A para un conjunto A 

def xA(A,O):
    
    s=size(D)
    
    matrizA=ones((5*s,4+3*s))
    K=0.0
    
    indicA=zeros(s)
    for i in range(size(A)):
        cual=D.index(A[i])
        indicA[cual]=1

    epsilon=gamma(A)

    with mosek.Env() as env:
        with env.Task(0, 0) as task:
        
            # Bound keys for constraints
            bkc = [mosek.boundkey.lo]*4*s+[mosek.boundkey.fx]*s

            # Bound values for constraints

            blc = [0]*5*s
            buc = [inf]*4*s+[0]*s

            # Bound keys for variables
            bkx = [mosek.boundkey.fr]*2+[mosek.boundkey.lo]*(2*s+1)+[mosek.boundkey.ra]*s+[mosek.boundkey.lo]

            # Bound values for variables
            blx = [-inf]*2+[0]*(3*s+2)
            bux = [+inf]*(3+2*s)+D+[+inf]

            # Objective coefficients
            mO=[]
            for i in range(size(O)):
                mO.append(-O[i])            
            
            coef = [1]+[0]+[epsilon]+O+mO+[K]*s+[c]

            # Below is the sparse representation of the A
            # matrix stored by column.

            numvar = len(bkx)
            numcon = len(bkc)

            # Append 'numcon' empty constraints.
            # The constraints will initially have no bounds.
            task.appendcons(numcon)

            # Append 'numvar' variables.
            # The variables will initially be fixed at zero (x=0).
            task.appendvars(numvar)

            #Append constraint matrix 
            task.putaijlist(range(0,s),[0]*s,indicA)
            task.putaijlist(range(0,s),[1]*s,(ones(s)-indicA))
            task.putaijlist(range(0,s),range(3,3+s),ones(s))
            task.putaijlist(range(0,s),range(3+s,3+2*s),-ones(s)) 
            task.putaijlist(range(0,s),range(3+2*s,3+3*s),p*ones(s))
            task.putaijlist(range(s,2*s),[2]*s,ones(s))
            task.putaijlist(range(s,2*s),range(3,3+s),-ones(s))
            task.putaijlist(range(2*s,3*s),range(3+s,3+2*s),-ones(s))
            task.putaijlist(range(2*s,3*s),[2]*s,ones(s))
            task.putaijlist(range(3*s,4*s),range(3+2*s,3+3*s),-ones(s))
            task.putaijlist(range(3*s,4*s),[3+3*s]*s,ones(s))
            task.putaijlist(range(4*s,5*s),range(3+2*s,3+3*s),(ones(s)-indicA))
        
        
            for i in range(numcon):
                for j in range(numvar):
                    matrizA[i,j]=task.getaij(i,j) 

            for j in range(numvar):
                # Set the linear term c_j in the objective.
                task.putcj(j, coef[j])

                # Set the bounds on variable j
                # blx[j] <= x_j <= bux[j]
                task.putvarbound(j, bkx[j], blx[j], bux[j])  


            # Set the bounds on constraints.
            # blc[i] <= constraint_i <= buc[i]
            for i in range(numcon):
                task.putconbound(i, bkc[i], blc[i], buc[i])

            # Input the objective sense (minimize/maximize)
            task.putobjsense(mosek.objsense.minimize)

            # Solve the problem
            task.optimize()

            # Print a summary containing information
            # about the solution for debugging purposes
            task.solutionsummary(mosek.streamtype.msg)

            # Get status information about the solution
            solsta = task.getsolsta(mosek.soltype.bas)

            if (solsta == mosek.solsta.optimal or solsta == mosek.solsta.near_optimal): 
                xx = [0.] * numvar
                task.getxx(mosek.soltype.bas, xx) # Request the basic solution.
                sol=xx[numvar-1]
                
            elif (solsta == mosek.solsta.dual_infeas_cer ):
                print("Dual infeasibility certificate found.\n")
                sol="No factible"
            elif(solsta == mosek.solsta.prim_infeas_cer or solsta == mosek.solsta.near_prim_infeas_cer):
                print("Primal infeasibility certificate found.\n")
                sol="No factible"
            elif solsta == mosek.solsta.unknown:
                print("Unknown solution status")
                sol="No factible"
            else:
                print("Other solution status")
                sol="Other solution status"
    
    return sol


In [21]:
#Función que devuelve la solución x^* para un conjunto A con distribución Q

def xARed(A,Q):
    
    s=size(A)
    
    matrizA=ones((5*s,4+3*s))
    K=0.0
    
    indicA=ones(s)
    epsilon=2*eps

    with mosek.Env() as env:
        with env.Task(0, 0) as task:
        
            # Bound keys for constraints
            bkc = [mosek.boundkey.lo]*4*s+[mosek.boundkey.fx]*s

            # Bound values for constraints

            blc = [0]*5*s
            buc = [inf]*4*s+[0]*s

            # Bound keys for variables
            bkx = [mosek.boundkey.fr]*2+[mosek.boundkey.lo]*(2*s+1)+[mosek.boundkey.ra]*s+[mosek.boundkey.lo]

            # Bound values for variables
            blx = [-inf]*2+[0]*(3*s+2)
            bux = [+inf]*(3+2*s)+D+[+inf]

            # Objective coefficients
            mO=[]
            for i in range(size(Q)):
                mO.append(-Q[i])            
            
            coef = [1]+[0]+[epsilon]+Q+mO+[K]*s+[c]

            # Below is the sparse representation of the A
            # matrix stored by column.

            numvar = len(bkx)
            numcon = len(bkc)

            # Append 'numcon' empty constraints.
            # The constraints will initially have no bounds.
            task.appendcons(numcon)

            # Append 'numvar' variables.
            # The variables will initially be fixed at zero (x=0).
            task.appendvars(numvar)

            #Append constraint matrix 
            task.putaijlist(range(0,s),[0]*s,indicA)
            task.putaijlist(range(0,s),[1]*s,(ones(s)-indicA))
            task.putaijlist(range(0,s),range(3,3+s),ones(s))
            task.putaijlist(range(0,s),range(3+s,3+2*s),-ones(s)) 
            task.putaijlist(range(0,s),range(3+2*s,3+3*s),p*ones(s))
            task.putaijlist(range(s,2*s),[2]*s,ones(s))
            task.putaijlist(range(s,2*s),range(3,3+s),-ones(s))
            task.putaijlist(range(2*s,3*s),range(3+s,3+2*s),-ones(s))
            task.putaijlist(range(2*s,3*s),[2]*s,ones(s))
            task.putaijlist(range(3*s,4*s),range(3+2*s,3+3*s),-ones(s))
            task.putaijlist(range(3*s,4*s),[3+3*s]*s,ones(s))
            task.putaijlist(range(4*s,5*s),range(3+2*s,3+3*s),(ones(s)-indicA))
        
        
            for i in range(numcon):
                for j in range(numvar):
                    matrizA[i,j]=task.getaij(i,j) 

            for j in range(numvar):
                # Set the linear term c_j in the objective.
                task.putcj(j, coef[j])

                # Set the bounds on variable j
                # blx[j] <= x_j <= bux[j]
                task.putvarbound(j, bkx[j], blx[j], bux[j])  


            # Set the bounds on constraints.
            # blc[i] <= constraint_i <= buc[i]
            for i in range(numcon):
                task.putconbound(i, bkc[i], blc[i], buc[i])

            # Input the objective sense (minimize/maximize)
            task.putobjsense(mosek.objsense.minimize)

            # Solve the problem
            task.optimize()

            # Print a summary containing information
            # about the solution for debugging purposes
            task.solutionsummary(mosek.streamtype.msg)

            # Get status information about the solution
            solsta = task.getsolsta(mosek.soltype.bas)

            if (solsta == mosek.solsta.optimal or solsta == mosek.solsta.near_optimal): 
                xx = [0.] * numvar
                task.getxx(mosek.soltype.bas, xx) # Request the basic solution.
                sol=xx[numvar-1]
                
            elif (solsta == mosek.solsta.dual_infeas_cer ):
                print("Dual infeasibility certificate found.\n")
                sol="No factible"
            elif(solsta == mosek.solsta.prim_infeas_cer or solsta == mosek.solsta.near_prim_infeas_cer):
                print("Primal infeasibility certificate found.\n")
                sol="No factible"
            elif solsta == mosek.solsta.unknown:
                print("Unknown solution status")
                sol="No factible"
            else:
                print("Other solution status")
                sol="Other solution status"
    
    return sol

In [22]:
#Función que devuelve el valor óptimo del problema robusto para un conjunto A con distribución Q

def VOR(A,Q):
    
    s=size(A)
    
    matrizA=ones((5*s,4+3*s))
    K=0.0
    
    indicA=ones(s)
    epsilon=2*eps

    
    with mosek.Env() as env:
        with env.Task(0, 0) as task:
        
            # Bound keys for constraints
            bkc = [mosek.boundkey.lo]*4*s+[mosek.boundkey.fx]*s

            # Bound values for constraints

            blc = [0]*5*s
            buc = [inf]*4*s+[0]*s

            # Bound keys for variables
            bkx = [mosek.boundkey.fr]*2+[mosek.boundkey.lo]*(2*s+1)+[mosek.boundkey.ra]*s+[mosek.boundkey.lo]

            # Bound values for variables
            blx = [-inf]*2+[0]*(3*s+2)
            bux = [+inf]*(3+2*s)+D+[+inf]

            # Objective coefficients
            mO=[]
            for i in range(size(Q)):
                mO.append(-Q[i])            
            
            coef = [1]+[0]+[epsilon]+Q+mO+[K]*s+[c]

            # Below is the sparse representation of the A
            # matrix stored by column.

            numvar = len(bkx)
            numcon = len(bkc)

            # Append 'numcon' empty constraints.
            # The constraints will initially have no bounds.
            task.appendcons(numcon)

            # Append 'numvar' variables.
            # The variables will initially be fixed at zero (x=0).
            task.appendvars(numvar)

            #Append constraint matrix 
            task.putaijlist(range(0,s),[0]*s,indicA)
            task.putaijlist(range(0,s),[1]*s,(ones(s)-indicA))
            task.putaijlist(range(0,s),range(3,3+s),ones(s))
            task.putaijlist(range(0,s),range(3+s,3+2*s),-ones(s)) 
            task.putaijlist(range(0,s),range(3+2*s,3+3*s),p*ones(s))
            task.putaijlist(range(s,2*s),[2]*s,ones(s))
            task.putaijlist(range(s,2*s),range(3,3+s),-ones(s))
            task.putaijlist(range(2*s,3*s),range(3+s,3+2*s),-ones(s))
            task.putaijlist(range(2*s,3*s),[2]*s,ones(s))
            task.putaijlist(range(3*s,4*s),range(3+2*s,3+3*s),-ones(s))
            task.putaijlist(range(3*s,4*s),[3+3*s]*s,ones(s))
            task.putaijlist(range(4*s,5*s),range(3+2*s,3+3*s),(ones(s)-indicA))
        
        
            for i in range(numcon):
                for j in range(numvar):
                    matrizA[i,j]=task.getaij(i,j) 

            for j in range(numvar):
                # Set the linear term c_j in the objective.
                task.putcj(j, coef[j])

                # Set the bounds on variable j
                # blx[j] <= x_j <= bux[j]
                task.putvarbound(j, bkx[j], blx[j], bux[j])  


            # Set the bounds on constraints.
            # blc[i] <= constraint_i <= buc[i]
            for i in range(numcon):
                task.putconbound(i, bkc[i], blc[i], buc[i])

            # Input the objective sense (minimize/maximize)
            task.putobjsense(mosek.objsense.minimize)

            # Solve the problem
            task.optimize()

            # Print a summary containing information
            # about the solution for debugging purposes
            task.solutionsummary(mosek.streamtype.msg)

            # Get status information about the solution
            solsta = task.getsolsta(mosek.soltype.bas)

            if (solsta == mosek.solsta.optimal or solsta == mosek.solsta.near_optimal): 
                xx = [0.] * numvar
                task.getxx(mosek.soltype.bas, xx) # Request the basic solution.
                sol=task.getprimalobj(soltype.bas)
                
            elif (solsta == mosek.solsta.dual_infeas_cer ):
                print("Dual infeasibility certificate found.\n")
                sol="No factible"
            elif(solsta == mosek.solsta.prim_infeas_cer or solsta == mosek.solsta.near_prim_infeas_cer):
                print("Primal infeasibility certificate found.\n")
                sol="No factible"
            elif solsta == mosek.solsta.unknown:
                print("Unknown solution status")
                sol="No factible"
            else:
                print("Other solution status")
                sol="Other solution status"
    
    return sol



In [23]:
#Optimo no robusto: 

epstemp=eps

eps=0
xNR=xA(D,P)
optimoNR=VOR(D,P)

print(xNR)
print(optimoNR)

eps=epstemp

20.99999982879635
-124.9199985998273


**Clustering con la distancia 1

$d_1(A,B):=2f(X_{A\cup B}^*,A\cup B)-f(X_A^*,A)-f(X_B^*,B)$

In [24]:
#Tabla Distancia 1

colk=[]
s=size(D)
for i in range(s-1):
    colk.append(s-i-1)

COLE=[]
COLEDos=[]
epsilons=[]

for e in range(numeps):
    
    eps=e/(numeps-1)
    hhh='e='+str(eps)
    epsilons.append(hhh)
    
    cole=[]
    coleDos=[]

    M=zeros((s,s))
    Z=[]
    for i in range(s):
        Z.append([D[i]])

    xopt=xA(D,P)

    contador=size(Z)

    for i in range(size(Z)):
            for j in range(i,size(Z)):
                if i!=j:
                    AuB=Z[i]+Z[j]
                    A=Z[i]
                    B=Z[j]
                    xab=xA(AuB,P)
                    xa=xA(A,P)
                    xb=xA(B,P)
                    M[i,j]=2*funcionf(xab,AuB)-funcionf(xa,A)-funcionf(xb,B)
                    M[j,i]=M[i,j]


    while contador>1:


        m=1000000000
        im=0
        jm=0


        for i in range(contador):
            for j in range(i,contador):
                if i!=j and M[i,j]<m:
                    m=M[i,j]
                    im=i
                    jm=j

        Ztemp=[]
        for k in range(contador):
            if k!=im and k!=jm:
                Ztemp.append(Z[k])

        C=Z[im]+Z[jm]
        Ztemp.append(C)
        Z=Ztemp
        contador=contador-1

        rep=[]
        Q=[]

        for k in range(contador):
            pk=0
            for l in range(size(Z[k])):
                pk=pk+P[D.index(Z[k][l])]

            rep.append(min(Z[k]))
            Q.append(pk)

        rep.sort()


        vopt=VOR(D,P)
        medida1=abs(VOR(rep,Q)-vopt)/abs(vopt)
        cole.append(medida1)
        
        etemp=eps
        eps=0
        oh=VOR(rep,Q)
        eps=etemp
    
        obje=optimoNR
        medida2=abs(oh-obje)/abs(obje)
        coleDos.append(medida2)


        t=contador
        MV=M
        M=zeros((t,t))

        k=0
        for i in range(t+1):
            l=0
            if i!=im and i!=jm:
                for j in range((t+1)):
                    if j!=im and j!=jm:
                        M[k,l]=MV[i,j]
                        l=l+1
                k=k+1

        for i in range(t-1):
                AuB=Z[i]+Z[t-1]
                A=Z[i]
                B=Z[t-1]
                xab=xA(AuB,P)
                xa=xA(A,P)
                xb=xA(B,P)
                M[i,(t-1)]=2*funcionf(xab,AuB)-funcionf(xa,A)-funcionf(xb,B)
                M[(t-1),i]=M[i,(t-1)]
        
    COLE.append(cole)
    COLEDos.append(coleDos)
            

#Se imprime la tabla de la medida 1 

with open('distancia1_medida1.csv','w',newline='') as tabla:
    thewriter=csv.writer(tabla)
    
    thewriter.writerow(['k']+epsilons)
    
    for i in range(size(colk)):
        fila=[colk[i]]
        for j in range(numeps):
            fila.append(COLE[j][i])  
        thewriter.writerow(fila)

#Se imprime la tabla de la medida 2 

with open('distancia1_medida2.csv','w',newline='') as tabla:
    thewriter=csv.writer(tabla)
    
    thewriter.writerow(['k']+epsilons)
    
    for i in range(size(colk)):
        fila=[colk[i]]
        for j in range(numeps):
            fila.append(COLEDos[j][i])  
        thewriter.writerow(fila)
        


In [25]:
#Dendograma Distancia 1

eps=0.25
colDendo=[]
colAlturas=[]

M=zeros((s,s))
Z=[]
for i in range(s):
    Z.append([D[i]])

colDendo.append(D)
print(D)

xopt=xA(D,P)

contador=size(Z)

for i in range(size(Z)):
        for j in range(i,size(Z)):
            if i!=j:
                AuB=Z[i]+Z[j]
                A=Z[i]
                B=Z[j]
                xab=xA(AuB,P)
                xa=xA(A,P)
                xb=xA(B,P)
                M[i,j]=2*funcionf(xab,AuB)-funcionf(xa,A)-funcionf(xb,B)
                M[j,i]=M[i,j]
                               

while contador>1:
        
    
    m=1000000000
    im=0
    jm=0
    
    
    for i in range(contador):
        for j in range(i,contador):
            if i!=j and M[i,j]<m:
                m=M[i,j]
                im=i
                jm=j
                            
    colAlturas.append(abs(m))
    
    Ztemp=[]
    for k in range(contador):
        if k!=im and k!=jm:
            Ztemp.append(Z[k])
    
    C=Z[im]+Z[jm]
    Ztemp.append(C)
    Z=Ztemp
    contador=contador-1
    print("clusters: ",Z)
    
    col=zeros(s)
    ide=0
    for i in range(contador):
        ide=ide+1
        for j in range(size(Z[i])):
            cual=D.index(Z[i][j])
            col[cual]=ide
    
    colDendo.append(col)
    print(col)
    
    rep=[]
    Q=[]
    
    for k in range(contador):
        pk=0
        for l in range(size(Z[k])):
            pk=pk+P[D.index(Z[k][l])]
        
        rep.append(min(Z[k]))
        Q.append(pk)
    
    rep.sort()
    
    vopt=VOR(D,P)
    medida1=abs(VOR(rep,Q)-vopt)/abs(vopt)
  
    etemp=eps
    eps=0
    oh=VOR(rep,Q)
    eps=etemp
    
    obje=optimoNR
    medida2=abs(oh-obje)/abs(obje)


    t=contador
    MV=M
    M=zeros((t,t))
    
    k=0
    for i in range(t+1):
        l=0
        if i!=im and i!=jm:
            for j in range((t+1)):
                if j!=im and j!=jm:
                    M[k,l]=MV[i,j]
                    l=l+1
            k=k+1

    for i in range(t-1):
            AuB=Z[i]+Z[t-1]
            A=Z[i]
            B=Z[t-1]
            xab=xA(AuB,P)
            xa=xA(A,P)
            xb=xA(B,P)
            M[i,(t-1)]=2*funcionf(xab,AuB)-funcionf(xa,A)-funcionf(xb,B)
            M[(t-1),i]=M[i,(t-1)]
    


niveles=[]
for i in range(s):
    niveles.append('nivel'+str(i))
    
with open('dendodistancia1.csv','w',newline='') as tabla:
    thewriter=csv.writer(tabla)
    
    thewriter.writerow(niveles)
    
    for i in range(s):
        fila=[]
        for j in range(s):
            fila.append(colDendo[j][i])
        thewriter.writerow(fila)
    
with open('alturasdistancia1.csv','w',newline='') as tabla: 
    thewriter=csv.writer(tabla)
    thewriter.writerow(colAlturas)
    
print(colAlturas)
    


[9, 11, 12, 13, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 31]
clusters:  [[9], [13], [17], [18], [19], [20], [21], [22], [23], [24], [25], [26], [27], [28], [29], [31], [11, 12]]
[ 1. 17. 17.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16.]
clusters:  [[9], [13], [19], [20], [21], [22], [23], [24], [25], [26], [27], [28], [29], [31], [11, 12], [17, 18]]
[ 1. 15. 15.  2. 16. 16.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14.]
clusters:  [[9], [13], [21], [22], [23], [24], [25], [26], [27], [28], [29], [31], [11, 12], [17, 18], [19, 20]]
[ 1. 13. 13.  2. 14. 14. 15. 15.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12.]
clusters:  [[9], [13], [23], [24], [25], [26], [27], [28], [29], [31], [11, 12], [17, 18], [19, 20], [21, 22]]
[ 1. 11. 11.  2. 12. 12. 13. 13. 14. 14.  3.  4.  5.  6.  7.  8.  9. 10.]
clusters:  [[9], [13], [25], [26], [27], [28], [29], [31], [11, 12], [17, 18], [19, 20], [21, 22], [23, 24]]
[ 1.  9.  9.  2. 10. 10. 11. 11. 12. 12. 13. 13.  3.  4.  5.  6.  7

**Clustering con la distancia 2

$d_2(A,B):=f(X_{A\cup B}^*,B)-f(X_B^*,B)+f(X_{A\cup B}^*,A)-f(X_A^*,A)$

In [26]:
#Tabla Distancia 2

colk=[]
s=size(D)
for i in range(s-1):
    colk.append(s-i-1)

COLE=[]
COLEDos=[]
epsilons=[]

for e in range(numeps):
    
    eps=e/(numeps-1)
    hhh='e='+str(eps)
    epsilons.append(hhh)
    
    cole=[]
    coleDos=[]

    M=zeros((s,s))
    Z=[]
    for i in range(s):
        Z.append([D[i]])

    xopt=xA(D,P)

    contador=size(Z)

    for i in range(size(Z)):
            for j in range(i,size(Z)):
                if i!=j:
                    AuB=Z[i]+Z[j]
                    A=Z[i]
                    B=Z[j]
                    xab=xA(AuB,P)
                    xa=xA(A,P)
                    xb=xA(B,P)
                    M[i,j]=funcionf(xab,B)-funcionf(xb,B)+funcionf(xab,A)-funcionf(xa,A)
                    M[j,i]=M[i,j]


    while contador>1:


        m=1000000000
        im=0
        jm=0


        for i in range(contador):
            for j in range(i,contador):
                if i!=j and M[i,j]<m:
                    m=M[i,j]
                    im=i
                    jm=j

        Ztemp=[]
        for k in range(contador):
            if k!=im and k!=jm:
                Ztemp.append(Z[k])

        C=Z[im]+Z[jm]
        Ztemp.append(C)
        Z=Ztemp
        contador=contador-1

        rep=[]
        Q=[]

        for k in range(contador):
            pk=0
            for l in range(size(Z[k])):
                pk=pk+P[D.index(Z[k][l])]

            rep.append(min(Z[k]))
            Q.append(pk)

        rep.sort()

        vopt=VOR(D,P)
        medida1=abs(VOR(rep,Q)-vopt)/abs(vopt)
        cole.append(medida1)
        
        etemp=eps
        eps=0
        oh=VOR(rep,Q)
        eps=etemp
    
        obje=optimoNR
        medida2=abs(oh-obje)/abs(obje)
        coleDos.append(medida2)

        t=contador
        MV=M
        M=zeros((t,t))

        k=0
        for i in range(t+1):
            l=0
            if i!=im and i!=jm:
                for j in range((t+1)):
                    if j!=im and j!=jm:
                        M[k,l]=MV[i,j]
                        l=l+1
                k=k+1

        for i in range(t-1):
                AuB=Z[i]+Z[t-1]
                A=Z[i]
                B=Z[t-1]
                xab=xA(AuB,P)
                xa=xA(A,P)
                xb=xA(B,P)
                M[i,(t-1)]=funcionf(xab,B)-funcionf(xb,B)+funcionf(xab,A)-funcionf(xa,A)
                M[(t-1),i]=M[i,(t-1)]
        
    COLE.append(cole)
    COLEDos.append(coleDos)
            

#Se imprime la tabla de la medida 1 

with open('distancia2_medida1.csv','w',newline='') as tabla:
    thewriter=csv.writer(tabla)
    
    thewriter.writerow(['k']+epsilons)
    
    for i in range(size(colk)):
        fila=[colk[i]]
        for j in range(numeps):
            fila.append(COLE[j][i])  
        thewriter.writerow(fila)

#Se imprime la tabla de la medida 2 

with open('distancia2_medida2.csv','w',newline='') as tabla:
    thewriter=csv.writer(tabla)
    
    thewriter.writerow(['k']+epsilons)
    
    for i in range(size(colk)):
        fila=[colk[i]]
        for j in range(numeps):
            fila.append(COLEDos[j][i])  
        thewriter.writerow(fila)



In [27]:
#Dendograma Distancia 2

eps=0.25
colDendo=[]
colAlturas=[]

M=zeros((s,s))
Z=[]
for i in range(s):
    Z.append([D[i]])

colDendo.append(D)
print(D)

xopt=xA(D,P)

contador=size(Z)

for i in range(size(Z)):
        for j in range(i,size(Z)):
            if i!=j:
                AuB=Z[i]+Z[j]
                A=Z[i]
                B=Z[j]
                xab=xA(AuB,P)
                xa=xA(A,P)
                xb=xA(B,P)
                M[i,j]=funcionf(xab,B)-funcionf(xb,B)+funcionf(xab,A)-funcionf(xa,A)
                M[j,i]=M[i,j]
                               

while contador>1:
        
    
    m=1000000000
    im=0
    jm=0
    
    
    for i in range(contador):
        for j in range(i,contador):
            if i!=j and M[i,j]<m:
                m=M[i,j]
                im=i
                jm=j
                            
    colAlturas.append(abs(m))
    
    Ztemp=[]
    for k in range(contador):
        if k!=im and k!=jm:
            Ztemp.append(Z[k])
    
    C=Z[im]+Z[jm]
    Ztemp.append(C)
    Z=Ztemp
    contador=contador-1
    print("clusters: ",Z)
    
    col=zeros(s)
    ide=0
    for i in range(contador):
        ide=ide+1
        for j in range(size(Z[i])):
            cual=D.index(Z[i][j])
            col[cual]=ide
    
    colDendo.append(col)
    print(col)
    
    rep=[]
    Q=[]
    
    for k in range(contador):
        pk=0
        for l in range(size(Z[k])):
            pk=pk+P[D.index(Z[k][l])]
        
        rep.append(min(Z[k]))
        Q.append(pk)
    
    rep.sort()
    
    vopt=VOR(D,P)
    medida1=abs(VOR(rep,Q)-vopt)/abs(vopt)
  
    etemp=eps
    eps=0
    oh=VOR(rep,Q)
    eps=etemp
    
    obje=optimoNR
    medida2=abs(oh-obje)/abs(obje)

    t=contador
    MV=M
    M=zeros((t,t))
    
    k=0
    for i in range(t+1):
        l=0
        if i!=im and i!=jm:
            for j in range((t+1)):
                if j!=im and j!=jm:
                    M[k,l]=MV[i,j]
                    l=l+1
            k=k+1

    for i in range(t-1):
            AuB=Z[i]+Z[t-1]
            A=Z[i]
            B=Z[t-1]
            xab=xA(AuB,P)
            xa=xA(A,P)
            xb=xA(B,P)
            M[i,(t-1)]=funcionf(xab,B)-funcionf(xb,B)+funcionf(xab,A)-funcionf(xa,A)
            M[(t-1),i]=M[i,(t-1)]
    


niveles=[]
for i in range(s):
    niveles.append('nivel'+str(i))
    
with open('dendodistancia2.csv','w',newline='') as tabla:
    thewriter=csv.writer(tabla)
    
    thewriter.writerow(niveles)
    
    for i in range(s):
        fila=[]
        for j in range(s):
            fila.append(colDendo[j][i])
        thewriter.writerow(fila)
    
with open('alturasdistancia2.csv','w',newline='') as tabla: 
    thewriter=csv.writer(tabla)
    thewriter.writerow(colAlturas)
    
    

[9, 11, 12, 13, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 31]
clusters:  [[9], [11], [12], [13], [17], [18], [19], [20], [21], [22], [25], [26], [27], [28], [29], [31], [23, 24]]
[ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 17. 17. 11. 12. 13. 14. 15. 16.]
clusters:  [[9], [13], [17], [18], [19], [20], [21], [22], [25], [26], [27], [28], [29], [31], [23, 24], [11, 12]]
[ 1. 16. 16.  2.  3.  4.  5.  6.  7.  8. 15. 15.  9. 10. 11. 12. 13. 14.]
clusters:  [[9], [13], [19], [20], [21], [22], [25], [26], [27], [28], [29], [31], [23, 24], [11, 12], [17, 18]]
[ 1. 14. 14.  2. 15. 15.  3.  4.  5.  6. 13. 13.  7.  8.  9. 10. 11. 12.]
clusters:  [[9], [13], [21], [22], [25], [26], [27], [28], [29], [31], [23, 24], [11, 12], [17, 18], [19, 20]]
[ 1. 12. 12.  2. 13. 13. 14. 14.  3.  4. 11. 11.  5.  6.  7.  8.  9. 10.]
clusters:  [[9], [13], [25], [26], [27], [28], [29], [31], [23, 24], [11, 12], [17, 18], [19, 20], [21, 22]]
[ 1. 10. 10.  2. 11. 11. 12. 12. 13. 13.  9.  9.  3.  4.  5.  6.  7

**Clustering con la distancia 3

$d_3(A,B):=f(X_A^*,A\cup B)-f(X_A^*,A)+f(X_B^*,A\cup B)-f(X_B^*,B)$

In [28]:
#Tabla Distancia 3

colk=[]
s=size(D)
for i in range(s-1):
    colk.append(s-i-1)

COLE=[]
COLEDos=[]
epsilons=[]

for e in range(numeps):
    
    eps=e/(numeps-1)
    hhh='e='+str(eps)
    epsilons.append(hhh)
    
    cole=[]
    coleDos=[]

    M=zeros((s,s))
    Z=[]
    for i in range(s):
        Z.append([D[i]])

    xopt=xA(D,P)

    contador=size(Z)

    for i in range(size(Z)):
            for j in range(i,size(Z)):
                if i!=j:
                    AuB=Z[i]+Z[j]
                    A=Z[i]
                    B=Z[j]
                    xab=xA(AuB,P)
                    xa=xA(A,P)
                    xb=xA(B,P)
                    M[i,j]=funcionf(xb,AuB)-funcionf(xb,B)+funcionf(xa,AuB)-funcionf(xa,A)
                    M[j,i]=M[i,j]


    while contador>1:


        m=1000000000
        im=0
        jm=0


        for i in range(contador):
            for j in range(i,contador):
                if i!=j and M[i,j]<m:
                    m=M[i,j]
                    im=i
                    jm=j

        Ztemp=[]
        for k in range(contador):
            if k!=im and k!=jm:
                Ztemp.append(Z[k])

        C=Z[im]+Z[jm]
        Ztemp.append(C)
        Z=Ztemp
        contador=contador-1

        rep=[]
        Q=[]

        for k in range(contador):
            pk=0
            for l in range(size(Z[k])):
                pk=pk+P[D.index(Z[k][l])]

            rep.append(min(Z[k]))
            Q.append(pk)

        rep.sort()


        vopt=VOR(D,P)
        medida1=abs(VOR(rep,Q)-vopt)/abs(vopt)
        cole.append(medida1)
        
        etemp=eps
        eps=0
        oh=VOR(rep,Q)
        eps=etemp
    
        obje=optimoNR
        medida2=abs(oh-obje)/abs(obje)
        coleDos.append(medida2)


        t=contador
        MV=M
        M=zeros((t,t))

        k=0
        for i in range(t+1):
            l=0
            if i!=im and i!=jm:
                for j in range((t+1)):
                    if j!=im and j!=jm:
                        M[k,l]=MV[i,j]
                        l=l+1
                k=k+1

        for i in range(t-1):
                AuB=Z[i]+Z[t-1]
                A=Z[i]
                B=Z[t-1]
                xab=xA(AuB,P)
                xa=xA(A,P)
                xb=xA(B,P)
                M[i,(t-1)]=funcionf(xb,AuB)-funcionf(xb,B)+funcionf(xa,AuB)-funcionf(xa,A)
                M[(t-1),i]=M[i,(t-1)]
        
    COLE.append(cole)
    COLEDos.append(coleDos)
            

#Se imprime la tabla de la medida 1 

with open('distancia3_medida1.csv','w',newline='') as tabla:
    thewriter=csv.writer(tabla)
    
    thewriter.writerow(['k']+epsilons)
    
    for i in range(size(colk)):
        fila=[colk[i]]
        for j in range(numeps):
            fila.append(COLE[j][i])  
        thewriter.writerow(fila)

#Se imprime la tabla de la medida 2 

with open('distancia3_medida2.csv','w',newline='') as tabla:
    thewriter=csv.writer(tabla)
    
    thewriter.writerow(['k']+epsilons)
    
    for i in range(size(colk)):
        fila=[colk[i]]
        for j in range(numeps):
            fila.append(COLEDos[j][i])  
        thewriter.writerow(fila)




In [29]:
#Dendograma Distancia 3

eps=0.25
colDendo=[]
colAlturas=[]

M=zeros((s,s))
Z=[]
for i in range(s):
    Z.append([D[i]])

colDendo.append(D)
print(D)

xopt=xA(D,P)

contador=size(Z)

for i in range(size(Z)):
        for j in range(i,size(Z)):
            if i!=j:
                AuB=Z[i]+Z[j]
                A=Z[i]
                B=Z[j]
                xab=xA(AuB,P)
                xa=xA(A,P)
                xb=xA(B,P)
                M[i,j]=funcionf(xb,AuB)-funcionf(xb,B)+funcionf(xa,AuB)-funcionf(xa,A)
                M[j,i]=M[i,j]
                               

while contador>1:
        
    
    m=1000000000
    im=0
    jm=0
    
    
    for i in range(contador):
        for j in range(i,contador):
            if i!=j and M[i,j]<m:
                m=M[i,j]
                im=i
                jm=j
                            
    colAlturas.append(abs(m))
    
    Ztemp=[]
    for k in range(contador):
        if k!=im and k!=jm:
            Ztemp.append(Z[k])
    
    C=Z[im]+Z[jm]
    Ztemp.append(C)
    Z=Ztemp
    contador=contador-1
    print("clusters: ",Z)
    
    col=zeros(s)
    ide=0
    for i in range(contador):
        ide=ide+1
        for j in range(size(Z[i])):
            cual=D.index(Z[i][j])
            col[cual]=ide
    
    colDendo.append(col)
    print(col)
    
    rep=[]
    Q=[]
    
    for k in range(contador):
        pk=0
        for l in range(size(Z[k])):
            pk=pk+P[D.index(Z[k][l])]
        
        rep.append(min(Z[k]))
        Q.append(pk)
    
    rep.sort()
    
    vopt=VOR(D,P)
    medida1=abs(VOR(rep,Q)-vopt)/abs(vopt)
  
    etemp=eps
    eps=0
    oh=VOR(rep,Q)
    eps=etemp
    
    obje=optimoNR
    medida2=abs(oh-obje)/abs(obje)

    t=contador
    MV=M
    M=zeros((t,t))
    
    k=0
    for i in range(t+1):
        l=0
        if i!=im and i!=jm:
            for j in range((t+1)):
                if j!=im and j!=jm:
                    M[k,l]=MV[i,j]
                    l=l+1
            k=k+1

    for i in range(t-1):
            AuB=Z[i]+Z[t-1]
            A=Z[i]
            B=Z[t-1]
            xab=xA(AuB,P)
            xa=xA(A,P)
            xb=xA(B,P)
            M[i,(t-1)]=funcionf(xb,AuB)-funcionf(xb,B)+funcionf(xa,AuB)-funcionf(xa,A)
            M[(t-1),i]=M[i,(t-1)]
    


niveles=[]
for i in range(s):
    niveles.append('nivel'+str(i))
    
with open('dendodistancia3.csv','w',newline='') as tabla:
    thewriter=csv.writer(tabla)
    
    thewriter.writerow(niveles)
    
    for i in range(s):
        fila=[]
        for j in range(s):
            fila.append(colDendo[j][i])
        thewriter.writerow(fila)
    
with open('alturasdistancia3.csv','w',newline='') as tabla: 
    thewriter=csv.writer(tabla)
    thewriter.writerow(colAlturas)

[9, 11, 12, 13, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 31]
clusters:  [[9], [11], [12], [13], [17], [18], [21], [22], [23], [24], [25], [26], [27], [28], [29], [31], [19, 20]]
[ 1.  2.  3.  4.  5.  6. 17. 17.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16.]
clusters:  [[9], [11], [12], [13], [17], [21], [22], [23], [24], [25], [26], [27], [28], [29], [31], [18, 19, 20]]
[ 1.  2.  3.  4.  5. 16. 16. 16.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15.]
clusters:  [[9], [11], [12], [13], [21], [22], [23], [24], [25], [26], [27], [28], [29], [31], [17, 18, 19, 20]]
[ 1.  2.  3.  4. 15. 15. 15. 15.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14.]
clusters:  [[9], [11], [12], [13], [21], [22], [25], [26], [27], [28], [29], [31], [17, 18, 19, 20], [23, 24]]
[ 1.  2.  3.  4. 13. 13. 13. 13.  5.  6. 14. 14.  7.  8.  9. 10. 11. 12.]
clusters:  [[9], [11], [12], [13], [21], [25], [26], [27], [28], [29], [31], [17, 18, 19, 20], [22, 23, 24]]
[ 1.  2.  3.  4. 12. 12. 12. 12.  5. 13. 13. 13.  6.  7.  8.  9. 10

**Clustering con la distancia 4

$d_4(A,B):=f(X_A^*,A\cup B)+f(X_B^*,A\cup B)-2f(X_{A\cup B}^*,A\cup B)$

In [30]:
#Tabla Distancia 4

colk=[]
s=size(D)
for i in range(s-1):
    colk.append(s-i-1)

COLE=[]
COLEDos=[]
epsilons=[]

for e in range(numeps):
    
    eps=e/(numeps-1)
    hhh='e='+str(eps)
    epsilons.append(hhh)
    
    cole=[]
    coleDos=[]

    M=zeros((s,s))
    Z=[]
    for i in range(s):
        Z.append([D[i]])

    xopt=xA(D,P)

    contador=size(Z)

    for i in range(size(Z)):
            for j in range(i,size(Z)):
                if i!=j:
                    AuB=Z[i]+Z[j]
                    A=Z[i]
                    B=Z[j]
                    xab=xA(AuB,P)
                    xa=xA(A,P)
                    xb=xA(B,P)
                    M[i,j]=funcionf(xb,AuB)+funcionf(xa,AuB)-2*funcionf(xab,AuB)
                    M[j,i]=M[i,j]


    while contador>1:


        m=1000000000
        im=0
        jm=0


        for i in range(contador):
            for j in range(i,contador):
                if i!=j and M[i,j]<m:
                    m=M[i,j]
                    im=i
                    jm=j

        Ztemp=[]
        for k in range(contador):
            if k!=im and k!=jm:
                Ztemp.append(Z[k])

        C=Z[im]+Z[jm]
        Ztemp.append(C)
        Z=Ztemp
        contador=contador-1

        rep=[]
        Q=[]

        for k in range(contador):
            pk=0
            for l in range(size(Z[k])):
                pk=pk+P[D.index(Z[k][l])]

            rep.append(min(Z[k]))
            Q.append(pk)

        rep.sort()


        vopt=VOR(D,P)
        medida1=abs(VOR(rep,Q)-vopt)/abs(vopt)
        cole.append(medida1)
        
        etemp=eps
        eps=0
        oh=VOR(rep,Q)
        eps=etemp
    
        obje=optimoNR
        medida2=abs(oh-obje)/abs(obje)
        coleDos.append(medida2)


        t=contador
        MV=M
        M=zeros((t,t))

        k=0
        for i in range(t+1):
            l=0
            if i!=im and i!=jm:
                for j in range((t+1)):
                    if j!=im and j!=jm:
                        M[k,l]=MV[i,j]
                        l=l+1
                k=k+1

        for i in range(t-1):
                AuB=Z[i]+Z[t-1]
                A=Z[i]
                B=Z[t-1]
                xab=xA(AuB,P)
                xa=xA(A,P)
                xb=xA(B,P)
                M[i,(t-1)]=funcionf(xb,AuB)+funcionf(xa,AuB)-2*funcionf(xab,AuB)
                M[(t-1),i]=M[i,(t-1)]
        
    COLE.append(cole)
    COLEDos.append(coleDos)
            

#Se imprime la tabla de la medida 1 

with open('distancia4_medida1.csv','w',newline='') as tabla:
    thewriter=csv.writer(tabla)
    
    thewriter.writerow(['k']+epsilons)
    
    for i in range(size(colk)):
        fila=[colk[i]]
        for j in range(numeps):
            fila.append(COLE[j][i])  
        thewriter.writerow(fila)

#Se imprime la tabla de la medida 2 

with open('distancia4_medida2.csv','w',newline='') as tabla:
    thewriter=csv.writer(tabla)
    
    thewriter.writerow(['k']+epsilons)
    
    for i in range(size(colk)):
        fila=[colk[i]]
        for j in range(numeps):
            fila.append(COLEDos[j][i])  
        thewriter.writerow(fila)



In [31]:
#Dendograma Distancia 4

eps=0.25
colDendo=[]
colAlturas=[]

M=zeros((s,s))
Z=[]
for i in range(s):
    Z.append([D[i]])

colDendo.append(D)
print(D)

xopt=xA(D,P)

contador=size(Z)

for i in range(size(Z)):
        for j in range(i,size(Z)):
            if i!=j:
                AuB=Z[i]+Z[j]
                A=Z[i]
                B=Z[j]
                xab=xA(AuB,P)
                xa=xA(A,P)
                xb=xA(B,P)
                M[i,j]=funcionf(xb,AuB)+funcionf(xa,AuB)-2*funcionf(xab,AuB)
                M[j,i]=M[i,j]
                               

while contador>1:
        
    
    m=1000000000
    im=0
    jm=0
    
    
    for i in range(contador):
        for j in range(i,contador):
            if i!=j and M[i,j]<m:
                m=M[i,j]
                im=i
                jm=j
                            
    colAlturas.append(abs(m))
    
    Ztemp=[]
    for k in range(contador):
        if k!=im and k!=jm:
            Ztemp.append(Z[k])
    
    C=Z[im]+Z[jm]
    Ztemp.append(C)
    Z=Ztemp
    contador=contador-1
    print("clusters: ",Z)
    
    col=zeros(s)
    ide=0
    for i in range(contador):
        ide=ide+1
        for j in range(size(Z[i])):
            cual=D.index(Z[i][j])
            col[cual]=ide
    
    colDendo.append(col)
    print(col)
    
    rep=[]
    Q=[]
    
    for k in range(contador):
        pk=0
        for l in range(size(Z[k])):
            pk=pk+P[D.index(Z[k][l])]
        
        rep.append(min(Z[k]))
        Q.append(pk)
    
    rep.sort()
    
    vopt=VOR(D,P)
    medida1=abs(VOR(rep,Q)-vopt)/abs(vopt)
  
    etemp=eps
    eps=0
    oh=VOR(rep,Q)
    eps=etemp
    
    obje=optimoNR
    medida2=abs(oh-obje)/abs(obje)
    t=contador
    MV=M
    M=zeros((t,t))
    
    k=0
    for i in range(t+1):
        l=0
        if i!=im and i!=jm:
            for j in range((t+1)):
                if j!=im and j!=jm:
                    M[k,l]=MV[i,j]
                    l=l+1
            k=k+1

    for i in range(t-1):
            AuB=Z[i]+Z[t-1]
            A=Z[i]
            B=Z[t-1]
            xab=xA(AuB,P)
            xa=xA(A,P)
            xb=xA(B,P)
            M[i,(t-1)]=funcionf(xb,AuB)+funcionf(xa,AuB)-2*funcionf(xab,AuB)
            M[(t-1),i]=M[i,(t-1)]
    


niveles=[]
for i in range(s):
    niveles.append('nivel'+str(i))
    
with open('dendodistancia4.csv','w',newline='') as tabla:
    thewriter=csv.writer(tabla)
    
    thewriter.writerow(niveles)
    
    for i in range(s):
        fila=[]
        for j in range(s):
            fila.append(colDendo[j][i])
        thewriter.writerow(fila)
    
with open('alturasdistancia4.csv','w',newline='') as tabla: 
    thewriter=csv.writer(tabla)
    thewriter.writerow(colAlturas)

[9, 11, 12, 13, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 31]
clusters:  [[9], [11], [12], [13], [17], [18], [21], [22], [23], [24], [25], [26], [27], [28], [29], [31], [19, 20]]
[ 1.  2.  3.  4.  5.  6. 17. 17.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16.]
clusters:  [[9], [11], [12], [13], [17], [21], [22], [23], [24], [25], [26], [27], [28], [29], [31], [18, 19, 20]]
[ 1.  2.  3.  4.  5. 16. 16. 16.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15.]
clusters:  [[9], [11], [12], [13], [21], [22], [23], [24], [25], [26], [27], [28], [29], [31], [17, 18, 19, 20]]
[ 1.  2.  3.  4. 15. 15. 15. 15.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14.]
clusters:  [[9], [11], [12], [13], [21], [22], [25], [26], [27], [28], [29], [31], [17, 18, 19, 20], [23, 24]]
[ 1.  2.  3.  4. 13. 13. 13. 13.  5.  6. 14. 14.  7.  8.  9. 10. 11. 12.]
clusters:  [[9], [11], [12], [13], [21], [25], [26], [27], [28], [29], [31], [17, 18, 19, 20], [22, 23, 24]]
[ 1.  2.  3.  4. 12. 12. 12. 12.  5. 13. 13. 13.  6.  7.  8.  9. 10

**Clustering con la distancia 5

$d_5(A,B):=f(X_A^*,B)-f(X_B^*,B)+f(X_B^*,A)-f(X_A^*,A)$

In [32]:
#Tabla Distancia 5

colk=[]
s=size(D)
for i in range(s-1):
    colk.append(s-i-1)

COLE=[]
COLEDos=[]
epsilons=[]

for e in range(numeps):
    
    eps=e/(numeps-1)
    hhh='e='+str(eps)
    epsilons.append(hhh)
    
    cole=[]
    coleDos=[]

    M=zeros((s,s))
    Z=[]
    for i in range(s):
        Z.append([D[i]])

    xopt=xA(D,P)

    contador=size(Z)

    for i in range(size(Z)):
            for j in range(i,size(Z)):
                if i!=j:
                    AuB=Z[i]+Z[j]
                    A=Z[i]
                    B=Z[j]
                    xab=xA(AuB,P)
                    xa=xA(A,P)
                    xb=xA(B,P)
                    M[i,j]=funcionf(xa,B)-funcionf(xb,B)+funcionf(xb,A)-funcionf(xa,A)
                    M[j,i]=M[i,j]


    while contador>1:


        m=1000000000
        im=0
        jm=0


        for i in range(contador):
            for j in range(i,contador):
                if i!=j and M[i,j]<m:
                    m=M[i,j]
                    im=i
                    jm=j

        Ztemp=[]
        for k in range(contador):
            if k!=im and k!=jm:
                Ztemp.append(Z[k])

        C=Z[im]+Z[jm]
        Ztemp.append(C)
        Z=Ztemp
        contador=contador-1

        rep=[]
        Q=[]

        for k in range(contador):
            pk=0
            for l in range(size(Z[k])):
                pk=pk+P[D.index(Z[k][l])]

            rep.append(min(Z[k]))
            Q.append(pk)

        rep.sort()


        vopt=VOR(D,P)
        medida1=abs(VOR(rep,Q)-vopt)/abs(vopt)
        cole.append(medida1)
        
        etemp=eps
        eps=0
        oh=VOR(rep,Q)
        eps=etemp
    
        obje=optimoNR
        medida2=abs(oh-obje)/abs(obje)
        coleDos.append(medida2)
        
        t=contador
        MV=M
        M=zeros((t,t))

        k=0
        for i in range(t+1):
            l=0
            if i!=im and i!=jm:
                for j in range((t+1)):
                    if j!=im and j!=jm:
                        M[k,l]=MV[i,j]
                        l=l+1
                k=k+1

        for i in range(t-1):
                AuB=Z[i]+Z[t-1]
                A=Z[i]
                B=Z[t-1]
                xab=xA(AuB,P)
                xa=xA(A,P)
                xb=xA(B,P)
                M[i,(t-1)]=funcionf(xa,B)-funcionf(xb,B)+funcionf(xb,A)-funcionf(xa,A)
                M[(t-1),i]=M[i,(t-1)]
        
    COLE.append(cole)
    COLEDos.append(coleDos)
            

#Se imprime la tabla de la medida 1 

with open('distancia5_medida1.csv','w',newline='') as tabla:
    thewriter=csv.writer(tabla)
    
    thewriter.writerow(['k']+epsilons)
    
    for i in range(size(colk)):
        fila=[colk[i]]
        for j in range(numeps):
            fila.append(COLE[j][i])  
        thewriter.writerow(fila)

#Se imprime la tabla de la medida 2 

with open('distancia5_medida2.csv','w',newline='') as tabla:
    thewriter=csv.writer(tabla)
    
    thewriter.writerow(['k']+epsilons)
    
    for i in range(size(colk)):
        fila=[colk[i]]
        for j in range(numeps):
            fila.append(COLEDos[j][i])  
        thewriter.writerow(fila)


In [33]:
#Dendograma Distancia 5

eps=0.25
colDendo=[]
colAlturas=[]

M=zeros((s,s))
Z=[]
for i in range(s):
    Z.append([D[i]])

colDendo.append(D)
print(D)

xopt=xA(D,P)

contador=size(Z)

for i in range(size(Z)):
        for j in range(i,size(Z)):
            if i!=j:
                AuB=Z[i]+Z[j]
                A=Z[i]
                B=Z[j]
                xab=xA(AuB,P)
                xa=xA(A,P)
                xb=xA(B,P)
                M[i,j]=funcionf(xa,B)-funcionf(xb,B)+funcionf(xb,A)-funcionf(xa,A)
                M[j,i]=M[i,j]
                               

while contador>1:
        
    
    m=1000000000
    im=0
    jm=0
    
    
    for i in range(contador):
        for j in range(i,contador):
            if i!=j and M[i,j]<m:
                m=M[i,j]
                im=i
                jm=j
                            
    colAlturas.append(abs(m))
    
    Ztemp=[]
    for k in range(contador):
        if k!=im and k!=jm:
            Ztemp.append(Z[k])
    
    C=Z[im]+Z[jm]
    Ztemp.append(C)
    Z=Ztemp
    contador=contador-1
    print("clusters: ",Z)
    
    col=zeros(s)
    ide=0
    for i in range(contador):
        ide=ide+1
        for j in range(size(Z[i])):
            cual=D.index(Z[i][j])
            col[cual]=ide
    
    colDendo.append(col)
    print(col)
    
    rep=[]
    Q=[]
    
    for k in range(contador):
        pk=0
        for l in range(size(Z[k])):
            pk=pk+P[D.index(Z[k][l])]
        
        rep.append(min(Z[k]))
        Q.append(pk)
    
    rep.sort()
    
    vopt=VOR(D,P)
    medida1=abs(VOR(rep,Q)-vopt)/abs(vopt)
  
    etemp=eps
    eps=0
    oh=VOR(rep,Q)
    eps=etemp
    
    obje=optimoNR
    medida2=abs(oh-obje)/abs(obje)
    

    t=contador
    MV=M
    M=zeros((t,t))
    
    k=0
    for i in range(t+1):
        l=0
        if i!=im and i!=jm:
            for j in range((t+1)):
                if j!=im and j!=jm:
                    M[k,l]=MV[i,j]
                    l=l+1
            k=k+1

    for i in range(t-1):
            AuB=Z[i]+Z[t-1]
            A=Z[i]
            B=Z[t-1]
            xab=xA(AuB,P)
            xa=xA(A,P)
            xb=xA(B,P)
            M[i,(t-1)]=funcionf(xa,B)-funcionf(xb,B)+funcionf(xb,A)-funcionf(xa,A)
            M[(t-1),i]=M[i,(t-1)]
    


niveles=[]
for i in range(s):
    niveles.append('nivel'+str(i))
    
with open('dendodistancia5.csv','w',newline='') as tabla:
    thewriter=csv.writer(tabla)
    
    thewriter.writerow(niveles)
    
    for i in range(s):
        fila=[]
        for j in range(s):
            fila.append(colDendo[j][i])
        thewriter.writerow(fila)
    
with open('alturasdistancia5.csv','w',newline='') as tabla: 
    thewriter=csv.writer(tabla)
    thewriter.writerow(colAlturas)

[9, 11, 12, 13, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 31]
clusters:  [[9], [13], [17], [18], [19], [20], [21], [22], [23], [24], [25], [26], [27], [28], [29], [31], [11, 12]]
[ 1. 17. 17.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16.]
clusters:  [[9], [13], [19], [20], [21], [22], [23], [24], [25], [26], [27], [28], [29], [31], [11, 12], [17, 18]]
[ 1. 15. 15.  2. 16. 16.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14.]
clusters:  [[9], [13], [21], [22], [23], [24], [25], [26], [27], [28], [29], [31], [11, 12], [17, 18], [19, 20]]
[ 1. 13. 13.  2. 14. 14. 15. 15.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12.]
clusters:  [[9], [13], [23], [24], [25], [26], [27], [28], [29], [31], [11, 12], [17, 18], [19, 20], [21, 22]]
[ 1. 11. 11.  2. 12. 12. 13. 13. 14. 14.  3.  4.  5.  6.  7.  8.  9. 10.]
clusters:  [[9], [13], [25], [26], [27], [28], [29], [31], [11, 12], [17, 18], [19, 20], [21, 22], [23, 24]]
[ 1.  9.  9.  2. 10. 10. 11. 11. 12. 12. 13. 13.  3.  4.  5.  6.  7

**Clustering con la distancia 6

$d_6(A,B):=\text{max}\{d_5(\omega_1,\omega_2):\omega_1\in A, \omega_2 \in B\}$

In [34]:
#Tabla Distancia 6

colk=[]
s=size(D)
for i in range(s-1):
    colk.append(s-i-1)

COLE=[]
COLEDos=[]
epsilons=[]


for e in range(numeps):
    
    eps=e/(numeps-1)
    hhh='e='+str(eps)
    epsilons.append(hhh)
    
    cole=[]
    coleDos=[]
    
    Z=[]
    for i in range(s):
        Z.append([D[i]])


    D5=zeros((s,s))
    for i in range(s):
        for j in range(i,s):
            if i!=j:
                A=[D[i]]
                B=[D[j]]
                xa=xA(A,P)
                xb=xA(B,P)
                D5[i,j]=funcionf(xa,B)-funcionf(xb,B)+funcionf(xb,A)-funcionf(xa,A)
                D5[j,i]=D5[i,j]


    xopt=xA(D,P)

    contador=size(Z)

    while contador>1:

        m=1000000000
        im=0
        jm=0

        M=zeros((size(Z),size(Z)))
        for i in range(size(Z)):
            for j in range(i,size(Z)):
                if i!=j:
                    A=Z[i]
                    B=Z[j]
                    distancia=0
                    for k in range(size(A)):
                        for l in range(size(B)):
                            posa=D.index(A[k])
                            posb=D.index(B[l])
                            if D5[posa,posb]>distancia:
                                distancia=D5[posa,posb]
                                M[i,j]=distancia
                    M[j,i]=M[i,j]
                    if M[i,j]<m: 
                        m=M[i,j];
                        im=i
                        jm=j



        C=Z[im]+Z[jm]
        Ztemp=[C]
        for k in range(size(Z)):
            if k!=im and k!=jm:
                Ztemp.append(Z[k])

        Z=Ztemp
        contador=contador-1


        rep=[]
        Q=[]

        for k in range(contador):
            pk=0
            for l in range(size(Z[k])):
                pk=pk+P[D.index(Z[k][l])]

            rep.append(Z[k][0])
            Q.append(pk)

        rep.sort()

        vopt=VOR(D,P)
        medida1=abs(VOR(rep,Q)-vopt)/abs(vopt)
        cole.append(medida1)
        
        etemp=eps
        eps=0
        oh=VOR(rep,Q)
        eps=etemp
    
        obje=optimoNR
        medida2=abs(oh-obje)/abs(obje)
        coleDos.append(medida2)
    
    COLE.append(cole)
    COLEDos.append(coleDos)

#Se imprime la tabla de la medida 1 

with open('distancia6_medida1.csv','w',newline='') as tabla:
    thewriter=csv.writer(tabla)
    
    thewriter.writerow(['k']+epsilons)
    
    for i in range(size(colk)):
        fila=[colk[i]]
        for j in range(numeps):
            fila.append(COLE[j][i])  
        thewriter.writerow(fila)

#Se imprime la tabla de la medida 2 

with open('distancia6_medida2.csv','w',newline='') as tabla:
    thewriter=csv.writer(tabla)
    
    thewriter.writerow(['k']+epsilons)
    
    for i in range(size(colk)):
        fila=[colk[i]]
        for j in range(numeps):
            fila.append(COLEDos[j][i])  
        thewriter.writerow(fila)





In [35]:
#Dendograma Distancia 6

eps=0.25
colDendo=[]
colAlturas=[]

Z=[]
for i in range(s):
    Z.append([D[i]])
    
colDendo.append(D)
print(D)

    

D5=zeros((s,s))
for i in range(s):
    for j in range(i,s):
        if i!=j:
            A=[D[i]]
            B=[D[j]]
            xa=xA(A,P)
            xb=xA(B,P)
            D5[i,j]=funcionf(xa,B)-funcionf(xb,B)+funcionf(xb,A)-funcionf(xa,A)
            D5[j,i]=D5[i,j]
        

xopt=xA(D,P)

contador=size(Z)

while contador>1:
    
    m=1000000000
    im=0
    jm=0

    M=zeros((size(Z),size(Z)))
    for i in range(size(Z)):
        for j in range(i,size(Z)):
            if i!=j:
                A=Z[i]
                B=Z[j]
                distancia=0
                for k in range(size(A)):
                    for l in range(size(B)):
                        posa=D.index(A[k])
                        posb=D.index(B[l])
                        if D5[posa,posb]>distancia:
                            distancia=D5[posa,posb]
                            M[i,j]=distancia
                M[j,i]=M[i,j]
                if M[i,j]<m: 
                    m=M[i,j];
                    im=i
                    jm=j
              
    colAlturas.append(abs(m))
    
    C=Z[im]+Z[jm]
    Ztemp=[C]
    for k in range(size(Z)):
        if k!=im and k!=jm:
            Ztemp.append(Z[k])

    Z=Ztemp
    contador=contador-1
    print("clusters: ",Z)
    
    col=zeros(s)
    ide=0
    for i in range(contador):
        ide=ide+1
        for j in range(size(Z[i])):
            cual=D.index(Z[i][j])
            col[cual]=ide
    
    colDendo.append(col)
    print(col)
    
    
    rep=[]
    Q=[]
    
    for k in range(contador):
        pk=0
        for l in range(size(Z[k])):
            pk=pk+P[D.index(Z[k][l])]
        
        rep.append(Z[k][0])
        Q.append(pk)
    
    rep.sort()
    
    vopt=VOR(D,P)
    medida1=abs(VOR(rep,Q)-vopt)/abs(vopt)
  
    etemp=eps
    eps=0
    oh=VOR(rep,Q)
    eps=etemp
    
    obje=optimoNR
    medida2=abs(oh-obje)/abs(obje)
    
    niveles=[]
for i in range(s):
    niveles.append('nivel'+str(i))
    
with open('dendodistancia6.csv','w',newline='') as tabla:
    thewriter=csv.writer(tabla)
    
    thewriter.writerow(niveles)
    
    for i in range(s):
        fila=[]
        for j in range(s):
            fila.append(colDendo[j][i])
        thewriter.writerow(fila)
    
with open('alturasdistancia6.csv','w',newline='') as tabla: 
    thewriter=csv.writer(tabla)
    thewriter.writerow(colAlturas)
    


[9, 11, 12, 13, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 31]
clusters:  [[11, 12], [9], [13], [17], [18], [19], [20], [21], [22], [23], [24], [25], [26], [27], [28], [29], [31]]
[ 2.  1.  1.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.]
clusters:  [[17, 18], [11, 12], [9], [13], [19], [20], [21], [22], [23], [24], [25], [26], [27], [28], [29], [31]]
[ 3.  2.  2.  4.  1.  1.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16.]
clusters:  [[19, 20], [17, 18], [11, 12], [9], [13], [21], [22], [23], [24], [25], [26], [27], [28], [29], [31]]
[ 4.  3.  3.  5.  2.  2.  1.  1.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15.]
clusters:  [[21, 22], [19, 20], [17, 18], [11, 12], [9], [13], [23], [24], [25], [26], [27], [28], [29], [31]]
[ 5.  4.  4.  6.  3.  3.  2.  2.  1.  1.  7.  8.  9. 10. 11. 12. 13. 14.]
clusters:  [[23, 24], [21, 22], [19, 20], [17, 18], [11, 12], [9], [13], [25], [26], [27], [28], [29], [31]]
[ 6.  5.  5.  7.  4.  4.  3.  3.  2.  2.  1.  1.  8.  9. 10. 11. 12

**Clustering con la distancia 7

$d_7(A,B):=\dfrac{1}{|A||B|}\sum_{\omega_1\in A} \sum_{\omega_2 \in B}d_5(\omega_1,\omega_2)$

In [36]:
#Tabla Distancia 7

colk=[]
s=size(D)
for i in range(s-1):
    colk.append(s-i-1)

COLE=[]
COLEDos=[]
epsilons=[]


for e in range(numeps):
    
    eps=e/(numeps-1)
    hhh='e='+str(eps)
    epsilons.append(hhh)
    
    cole=[]
    coleDos=[]
    
    Z=[]
    for i in range(s):
        Z.append([D[i]])


    D5=zeros((s,s))
    for i in range(s):
        for j in range(i,s):
            if i!=j:
                A=[D[i]]
                B=[D[j]]
                xa=xA(A,P)
                xb=xA(B,P)
                D5[i,j]=funcionf(xa,B)-funcionf(xb,B)+funcionf(xb,A)-funcionf(xa,A)
                D5[j,i]=D5[i,j]


    xopt=xA(D,P)

    contador=size(Z)

    while contador>1:

        m=1000000000
        im=0
        jm=0

        M=zeros((size(Z),size(Z)))
        for i in range(size(Z)):
            for j in range(i,size(Z)):
                if i!=j:
                    A=Z[i]
                    B=Z[j]
                    distancia=0
                    for k in range(size(A)):
                        for l in range(size(B)):
                            posa=D.index(A[k])
                            posb=D.index(B[l])
                            distancia=distancia+D5[posa,posb]

                    dist=distancia/(size(A)*size(B))
                    M[i,j]=dist
                    M[j,i]=M[i,j]
                    if M[i,j]<m: 
                        m=M[i,j];
                        im=i
                        jm=j
              



        C=Z[im]+Z[jm]
        Ztemp=[C]
        for k in range(size(Z)):
            if k!=im and k!=jm:
                Ztemp.append(Z[k])

        Z=Ztemp
        contador=contador-1


        rep=[]
        Q=[]

        for k in range(contador):
            pk=0
            for l in range(size(Z[k])):
                pk=pk+P[D.index(Z[k][l])]

            rep.append(Z[k][0])
            Q.append(pk)

        rep.sort()

        vopt=VOR(D,P)
        medida1=abs(VOR(rep,Q)-vopt)/abs(vopt)
        cole.append(medida1)
        
        etemp=eps
        eps=0
        oh=VOR(rep,Q)
        eps=etemp
    
        obje=optimoNR
        medida2=abs(oh-obje)/abs(obje)
        coleDos.append(medida2)
        
        
    COLE.append(cole)
    COLEDos.append(coleDos)

#Se imprime la tabla de la medida 1 

with open('distancia7_medida1.csv','w',newline='') as tabla:
    thewriter=csv.writer(tabla)
    
    thewriter.writerow(['k']+epsilons)
    
    for i in range(size(colk)):
        fila=[colk[i]]
        for j in range(numeps):
            fila.append(COLE[j][i])  
        thewriter.writerow(fila)

#Se imprime la tabla de la medida 2 

with open('distancia7_medida2.csv','w',newline='') as tabla:
    thewriter=csv.writer(tabla)
    
    thewriter.writerow(['k']+epsilons)
    
    for i in range(size(colk)):
        fila=[colk[i]]
        for j in range(numeps):
            fila.append(COLEDos[j][i])  
        thewriter.writerow(fila)





In [37]:
#Dendograma Distancia 7

eps=0.25
colDendo=[]
colAlturas=[]

Z=[]
for i in range(s):
    Z.append([D[i]])
    
colDendo.append(D)
print(D)

    

D5=zeros((s,s))
for i in range(s):
    for j in range(i,s):
        if i!=j:
            A=[D[i]]
            B=[D[j]]
            xa=xA(A,P)
            xb=xA(B,P)
            D5[i,j]=funcionf(xa,B)-funcionf(xb,B)+funcionf(xb,A)-funcionf(xa,A)
            D5[j,i]=D5[i,j]
        

xopt=xA(D,P)

contador=size(Z)

while contador>1:
    
    m=1000000000
    im=0
    jm=0

    M=zeros((size(Z),size(Z)))
    for i in range(size(Z)):
        for j in range(i,size(Z)):
            if i!=j:
                A=Z[i]
                B=Z[j]
                distancia=0
                for k in range(size(A)):
                     for l in range(size(B)):
                            posa=D.index(A[k])
                            posb=D.index(B[l])
                            distancia=distancia+D5[posa,posb]

                dist=distancia/(size(A)*size(B))
                M[i,j]=dist
                M[j,i]=M[i,j]
                if M[i,j]<m: 
                    m=M[i,j];
                    im=i
                    jm=j
                               
    colAlturas.append(abs(m))
    
    C=Z[im]+Z[jm]
    Ztemp=[C]
    for k in range(size(Z)):
        if k!=im and k!=jm:
            Ztemp.append(Z[k])

    Z=Ztemp
    contador=contador-1
    print("clusters: ",Z)
    
    col=zeros(s)
    ide=0
    for i in range(contador):
        ide=ide+1
        for j in range(size(Z[i])):
            cual=D.index(Z[i][j])
            col[cual]=ide
    
    colDendo.append(col)
    print(col)
    
    
    rep=[]
    Q=[]
    
    for k in range(contador):
        pk=0
        for l in range(size(Z[k])):
            pk=pk+P[D.index(Z[k][l])]
        
        rep.append(Z[k][0])
        Q.append(pk)
    
    rep.sort()
    
    vopt=VOR(D,P)
    medida1=abs(VOR(rep,Q)-vopt)/abs(vopt)
  
    etemp=eps
    eps=0
    oh=VOR(rep,Q)
    eps=etemp
    
    obje=optimoNR
    medida2=abs(oh-obje)/abs(obje)
    
    niveles=[]
for i in range(s):
    niveles.append('nivel'+str(i))
    
with open('dendodistancia7.csv','w',newline='') as tabla:
    thewriter=csv.writer(tabla)
    
    thewriter.writerow(niveles)
    
    for i in range(s):
        fila=[]
        for j in range(s):
            fila.append(colDendo[j][i])
        thewriter.writerow(fila)
    
with open('alturasdistancia7.csv','w',newline='') as tabla: 
    thewriter=csv.writer(tabla)
    thewriter.writerow(colAlturas)

[9, 11, 12, 13, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 31]
clusters:  [[11, 12], [9], [13], [17], [18], [19], [20], [21], [22], [23], [24], [25], [26], [27], [28], [29], [31]]
[ 2.  1.  1.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.]
clusters:  [[17, 18], [11, 12], [9], [13], [19], [20], [21], [22], [23], [24], [25], [26], [27], [28], [29], [31]]
[ 3.  2.  2.  4.  1.  1.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16.]
clusters:  [[19, 20], [17, 18], [11, 12], [9], [13], [21], [22], [23], [24], [25], [26], [27], [28], [29], [31]]
[ 4.  3.  3.  5.  2.  2.  1.  1.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15.]
clusters:  [[21, 22], [19, 20], [17, 18], [11, 12], [9], [13], [23], [24], [25], [26], [27], [28], [29], [31]]
[ 5.  4.  4.  6.  3.  3.  2.  2.  1.  1.  7.  8.  9. 10. 11. 12. 13. 14.]
clusters:  [[23, 24], [21, 22], [19, 20], [17, 18], [11, 12], [9], [13], [25], [26], [27], [28], [29], [31]]
[ 6.  5.  5.  7.  4.  4.  3.  3.  2.  2.  1.  1.  8.  9. 10. 11. 12